In [1]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [2]:
loader = PDFPlumberLoader(r"D:\Petramount\Courses\AI\LLM\1+The+structured+Prompt.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[1].page_content

Number of pages in the PDF: 2


'Analysis of the Example:\n• Modifier - Blog Post: Here, we are instructing the Language Model to create a\nmore detailed and in-depth text, as opposed to short, concise tweets.\n• Topic - Healthy Eating: This is the main focus of our request.\n• Target Audience - Working Professionals: The model needs to present the\ninformation in a way that is practical and actionable for working professionals\nwho may have limited time.\n• Use of Keywords - SEO Relevance: The model should incorporate keywords\nand phrases that can enhance visibility on search engines.\n• Style - Simple, Understandable Style: The model should make the text clear\nand comprehensible to facilitate easy reading.\n• Length and Structure - 800 Words, Well-Structured: These requirements help\ndetermine the length and formatting of the text.\n'

In [3]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

c:\Users\deepa\.conda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Number of chunks created: ", len(documents))

print(documents[0].page_content)

Number of chunks created:  4
Structured Prompt! A structured prompt consists of a (Modifier) + (Topic) + (Multiple Modifiers). These elements together form the optimized prompt. Modifier: Specifies the type of desired response. Examples: "Twitter thread,"
"blog post," "research paper," etc. Topic: The main subject or question to be addressed in the response. Additional Modifiers: Additional specific requirements or details to be
considered in the response. Examples: Target audience, keywords used, style,
length, structure, etc.


In [8]:
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b")

In [26]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

In [27]:
print(qa("prompting example")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:Structured Prompt! A structured prompt consists of a (Modifier) + (Topic) + (Multiple Modifiers). These elements together form the optimized prompt. Modifier: Specifies the type of desired response. Examples: "Twitter thread,"
"blog post," "research paper," etc. Topic: The main subject or question to be addressed in the response. Additional Modifiers: Additional specific requirements or details to be
considered in the response. Examples: Target audience, keywords used, style,
length, structure, etc.
source:D:\Petramount\Courses\AI\LLM\1+The+structured+Prompt.pdf

Context:
content:Analysis of the Example:
• Modifier - Blog Post: Here,

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")